In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(r'C:\Users\Dell\Documents\dsaiProject\data.csv',sep='\t')
df.info()

In [4]:
DASS_keys = {
    'Depression': [3, 5, 10, 13, 16, 17, 21, 24, 26, 31, 34, 37, 38, 42],
    'Anxiety': [2, 4, 7, 9, 15, 19, 20, 23, 25, 28, 30, 36, 40, 41],
    'Stress': [1, 6, 8, 11, 12, 14, 18, 22, 27, 29, 32, 33, 35, 39]
    }

In [ ]:
df['wrongansw'] = 0 
# This line creates a new column in the DataFrame called 'wrongansw' and initializes it to zero for all rows.

df.wrongansw.value_counts()

In [4]:
df['wrongansw'] = 0
df['wrongansw'] = df['wrongansw'].where(df['VCL6']== 0, df['wrongansw'] + 1)

# This line updates the 'wrongansw' column based on a condition. 
# It uses the where method of the DataFrame to apply a condition where df['VCL6'] equals 0. 
# If the condition is true, then the 'wrongansw' column remains unchanged. 
# If the condition is false, then the 'wrongansw' column is incremented by 1.

df.wrongansw.value_counts()

0    38177
1     1598
Name: wrongansw, dtype: int64

In [5]:
df['wrongansw'] = 0
df['wrongansw'] = df['wrongansw'].where(df['VCL6']== 0, df['wrongansw'] + 1)
df['wrongansw'] = df['wrongansw'].where(df['VCL9']== 0, df['wrongansw'] + 1)

df.wrongansw.value_counts()

0    36913
1     2410
2      452
Name: wrongansw, dtype: int64

In [6]:
df['wrongansw'] = 0
df['wrongansw'] = df['wrongansw'].where(df['VCL6']== 0, df['wrongansw'] + 1)
df['wrongansw'] = df['wrongansw'].where(df['VCL9']== 0, df['wrongansw'] + 1)
df['wrongansw'] = df['wrongansw'].where(df['VCL12']== 0, df['wrongansw'] + 1)

df.wrongansw.value_counts()

0    34583
1     4044
2      822
3      326
Name: wrongansw, dtype: int64

In [7]:
# Steps for data cleaning; VCL6, VCL9, VCL12 are not real words, used for validity check.
df = df[df['wrongansw'].isin([2, 3])]
df = df.drop(columns='wrongansw')
df.head(3)
df.shape

(1148, 172)

In [8]:
# removing the validity check columns
vcls = []
for i in range(1, 17):
    vcls.append('VCL' + str(i))
    
df = df.drop(columns=vcls)
df.shape

(1148, 156)

In [9]:
depr = []
for i in DASS_keys["Depression"]:
    depr.append('Q' + str(i) + 'A')
    
anx = []
for i in DASS_keys["Anxiety"]:
    anx.append('Q' + str(i) + 'A')

stre = []
for i in DASS_keys["Stress"]:
    stre.append('Q' + str(i) + 'A')

# making a new dataframe for each disorder 

df_depr=df.filter(depr) 
df_anx=df.filter(anx)
df_stre=df.filter(stre)

In [10]:
categorical = df.select_dtypes('object').columns

print('Categorical Columns: ', df[categorical].columns)
print(df[categorical].nunique())

# remove 'major' column
df = df.drop(['major'], axis=1)

Categorical Columns:  Index(['country', 'major'], dtype='object')
country     68
major      366
dtype: int64


In [11]:
disorders = [depr, anx, stre]

def scores(df): # adds new columns to store total scores for each        
    df["ScoresDepr"] = df[depr].sum(axis=1) 
    df["ScoresAnx"] = df[anx].sum(axis=1)
    df["ScoresStre"] = df[stre].sum(axis=1)
    return df

for i in disorders:
        df[i] -= 1 # to get the scores
        
df = scores(df)
        
df.head(3)

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,hand,religion,orientation,race,voted,married,familysize,ScoresDepr,ScoresAnx,ScoresStre
26,3,1,4820,3,39,1109,3,8,1833,3,...,1,7,1,60,2,1,3,38,41,41
31,1,25,3671,1,26,5743,1,13,3154,0,...,1,2,1,60,2,1,3,30,8,7
57,1,27,3827,2,33,6293,1,22,12153,2,...,1,1,1,60,1,1,2,13,15,20


In [12]:
# ANXIETY MODEL FROM NOW #

In [13]:
CategoryAnx=[]

for i in df['ScoresAnx']:
    if i in range(0,10):
        CategoryAnx.append('None')
    elif i in range(10,14):
        CategoryAnx.append('Minimal')
    elif i in range(14,21):
        CategoryAnx.append('Moderate')
    elif i in range(21,28):
        CategoryAnx.append('Severe')
    else:
        CategoryAnx.append('Extremely Severe')
        
df['CategoryAnx']= CategoryAnx

In [14]:
df.isnull().sum()
df.duplicated().sum()

0

In [15]:
Y = df['CategoryAnx']
X = df.drop(columns=['Q2A','Q4A','Q7A','Q9A','Q15A','Q19A','Q20A','Q23A','Q25A','Q28A','Q30A','Q36A','Q40A','Q41A','introelapse','testelapse', 'surveyelapse','engnat','CategoryAnx','country', 'ScoresDepr' ,'ScoresStre','screensize','uniquenetworklocation','Q1A' ,'Q1I', 'Q1E', 'Q2I', 'Q2E', 'Q3A', 'Q3I', 'Q3E','Q4I','Q4E', 'Q5E', 'Q5A' ,'Q5I', 'Q6E', 'Q6A' ,'Q6I', 'Q7E' ,'Q7I', 'Q8E','Q8A' ,'Q8I', 'Q9E', 'Q9I', 'Q10E', 'Q10A', 'Q10I', 'Q11E', 'Q11A' ,'Q11I', 'Q12E', 'Q12A' ,'Q12I', 'Q13E', 'Q13A' ,'Q13I', 'Q14E', 'Q14A' ,'Q14I', 'Q15E', 'Q15I', 'Q16E', 'Q16A' ,'Q16I', 'Q17E', 'Q17A' ,'Q17I', 'Q18E', 'Q18A' ,'Q18I', 'Q19I', 'Q19E',  'Q20I', 'Q20E', 'Q21A', 'Q21I', 'Q21E', 'Q22A', 'Q22I','Q22E', 'Q23I','Q23E','Q24A','Q24I','Q24E', 'Q25I', 'Q25E', 'Q26A', 'Q26I', 'Q26E', 'Q27A', 'Q27I', 'Q27E', 'Q28I', 'Q28E', 'Q29A', 'Q29I', 'Q29E',  'Q30I', 'Q30E', 'Q31A', 'Q31I', 'Q31E', 'Q32A', 'Q32I', 'Q32E', 'Q33A', 'Q33I', 'Q33E', 'Q34A', 'Q34I', 'Q34E', 'Q35A', 'Q35I', 'Q35E', 'Q36I', 'Q36E', 'Q37A', 'Q37I', 'Q37E', 'Q38A', 'Q38I', 'Q38E', 'Q39A', 'Q39I', 'Q39E', 'Q40I', 'Q40E', 'Q41I', 'Q41E', 'Q42A', 'Q42I', 'Q42E'])
pd.set_option("display.max_columns", None)
X.head()

,source,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,education,urban,gender,age,hand,religion,orientation,race,voted,married,familysize,ScoresAnx
26,2,4,1,7,7,7,2,6,5,1,1,1,1,2,15,1,7,1,60,2,1,3,41
31,2,5,5,5,5,7,7,7,5,5,6,2,3,1,18,1,2,1,60,2,1,3,8
57,0,2,4,1,5,6,4,3,6,6,3,3,3,2,21,1,1,1,60,1,1,2,15
81,2,7,6,3,6,4,1,7,5,3,2,1,2,2,14,1,4,1,70,2,1,6,10
87,2,1,3,5,6,6,7,2,7,1,2,2,3,1,17,1,1,1,30,2,1,3,36


In [16]:
#Pre-processing the data
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
print(X)

[[ 1.01193533  0.11146893 -1.80112123 ... -0.41158476 -0.04026184
   1.87354896]
 [ 1.01193533  0.59254537  0.33712817 ... -0.41158476 -0.04026184
  -0.86024639]
 [-1.3982343  -0.85068395 -0.19743418 ... -0.41158476 -0.41603906
  -0.28035041]
 ...
 [-1.3982343  -0.85068395 -0.73199653 ... -2.34805733  1.46284701
  -1.44014238]
 [-0.19314948 -1.33176039 -1.80112123 ...  3.46136038 -0.04026184
   0.63091471]
 [ 1.01193533 -0.36960751 -0.19743418 ... -0.41158476  0.71129258
   1.62502211]]


In [17]:
# KNN Model for Anxiety Classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

# Create a KNN classifier with k (number of neighbors)
knn = KNeighborsClassifier(n_neighbors=95)

# Fit the model to the training data
knn.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = knn.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)

print('Accuracy of KNN Model - Anxiety: {:.5f}'.format(accuracy))
print('Confusion Matrix of KNN Model - Anxiety: \n', conf_matrix)
print('Classification Report of KNN Model - Anxiety: \n', classif_report)

Accuracy of KNN Model - Anxiety: 0.66202
Confusion Matrix of KNN Model - Anxiety: 
 [[66  0  0  1  7]
 [ 1  0  6 16  3]
 [10  0 12 15 16]
 [ 1  0  4 97  4]
 [ 8  0  3  2 15]]


C:\Users\Dell\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [18]:
# Naive Bayes Model for Anxiety Classification
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

clfNB = GaussianNB()

# Fit the model to the training data
clfNB.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = clfNB.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)

print('Accuracy of NB Model - Anxiety: {:.5f}'.format(accuracy))
print('Confusion Matrix of NB Model - Anxiety: \n', conf_matrix)
print('Classification Report of KNN Model - Anxiety: \n', classif_report)

Accuracy of NB Model - Anxiety: 0.96864
Confusion Matrix of NB Model - Anxiety: 
 [[ 73   0   0   0   1]
 [  0  24   0   2   0]
 [  0   1  50   1   1]
 [  1   1   0 104   0]
 [  1   0   0   0  27]]


In [19]:
# Random Forest Classifier Model for Anxiety Classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

clfRFC = RandomForestClassifier(n_estimators=95, random_state=85)

# Fit the model to the training data
clfRFC.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = clfRFC.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)


print('Accuracy of RFC Model - Anxiety: {:.5f}'.format(accuracy))
print('Confusion Matrix of RFC Model - Anxiety: \n', conf_matrix)
print('Classification report of KNN Model - Anxiety: \n', classif_report)

Accuracy of RFC Model - Anxiety: 0.98955
Confusion Matrix of RFC Model - Anxiety: 
 [[ 74   0   0   0   0]
 [  0  23   3   0   0]
 [  0   0  53   0   0]
 [  0   0   0 106   0]
 [  0   0   0   0  28]]


In [ ]:
# DEPRESSION MODEL FROM NOW #

In [ ]:
CategoryDepr=[]

for i in df['ScoresDepr']:
    if i in range(0,10):
        CategoryDepr.append('None')
    elif i in range(10,14):
        CategoryDepr.append('Minimal')
    elif i in range(14,21):
        CategoryDepr.append('Moderate')
    elif i in range(21,28):
        CategoryDepr.append('Severe')
    else:
        CategoryDepr.append('Extremely Severe')
        
df['CategoryDepr']= CategoryDepr

In [ ]:
df.isnull().sum()
df.duplicated().sum()

In [ ]:
Y = df['CategoryDepr']
X = df.drop(columns=['Q2A','Q4A','Q7A','Q9A','Q15A','Q19A','Q20A','Q23A','Q25A','Q28A','Q30A','Q36A','Q40A','Q41A','introelapse','testelapse', 'surveyelapse','engnat','CategoryDepr','country', 'ScoresDepr' ,'ScoresStre','screensize','uniquenetworklocation','Q1A' ,'Q1I', 'Q1E', 'Q2I', 'Q2E', 'Q3A', 'Q3I', 'Q3E','Q4I','Q4E', 'Q5E', 'Q5A' ,'Q5I', 'Q6E', 'Q6A' ,'Q6I', 'Q7E' ,'Q7I', 'Q8E','Q8A' ,'Q8I', 'Q9E', 'Q9I', 'Q10E', 'Q10A', 'Q10I', 'Q11E', 'Q11A' ,'Q11I', 'Q12E', 'Q12A' ,'Q12I', 'Q13E', 'Q13A' ,'Q13I', 'Q14E', 'Q14A' ,'Q14I', 'Q15E', 'Q15I', 'Q16E', 'Q16A' ,'Q16I', 'Q17E', 'Q17A' ,'Q17I', 'Q18E', 'Q18A' ,'Q18I', 'Q19I', 'Q19E',  'Q20I', 'Q20E', 'Q21A', 'Q21I', 'Q21E', 'Q22A', 'Q22I','Q22E', 'Q23I','Q23E','Q24A','Q24I','Q24E', 'Q25I', 'Q25E', 'Q26A', 'Q26I', 'Q26E', 'Q27A', 'Q27I', 'Q27E', 'Q28I', 'Q28E', 'Q29A', 'Q29I', 'Q29E',  'Q30I', 'Q30E', 'Q31A', 'Q31I', 'Q31E', 'Q32A', 'Q32I', 'Q32E', 'Q33A', 'Q33I', 'Q33E', 'Q34A', 'Q34I', 'Q34E', 'Q35A', 'Q35I', 'Q35E', 'Q36I', 'Q36E', 'Q37A', 'Q37I', 'Q37E', 'Q38A', 'Q38I', 'Q38E', 'Q39A', 'Q39I', 'Q39E', 'Q40I', 'Q40E', 'Q41I', 'Q41E', 'Q42A', 'Q42I', 'Q42E'])
pd.set_option("display.max_columns", None)
X.head()

In [ ]:
#Pre-processing the data
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
print(X)

In [ ]:
# KNN Model for Depression Classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

# Create a KNN classifier with k (number of neighbors)
knn = KNeighborsClassifier(n_neighbors=95)

# Fit the model to the training data
knn.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = knn.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)

print('Accuracy of KNN Model - Depression: {:.5f}'.format(accuracy))
print('Confusion Matrix of KNN Model - Depression: \n', conf_matrix)
print('Confusion Matrix of KNN Model - Depression: \n', classif_report)

In [ ]:
# Naive Bayes Model for Depression Classification
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

clfNB = GaussianNB()

# Fit the model to the training data
clfNB.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = clfNB.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)

print('Accuracy of NB Model - Depression: {:.5f}'.format(accuracy))
print('Confusion Matrix of NB Model - Depression: \n', conf_matrix)
print('Classification Report of KNN Model - Depression: \n', classif_report)

In [ ]:
# Random Forest Classifier Model for Depression Classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

clfRFC = RandomForestClassifier(n_estimators=95, random_state=85)

# Fit the model to the training data
clfRFC.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = clfRFC.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)


print('Accuracy of RFC Model - Depression: {:.5f}'.format(accuracy))
print('Confusion Matrix of RFC Model - Depression: \n', conf_matrix)
print('Classification report of KNN Model - Depression: \n', classif_report)

In [ ]:
# STRESS MODEL FROM NOW #

In [ ]:
CategoryStre=[]

for i in df['ScoresStre']:
    if i in range(0,10):
        CategoryStre.append('None')
    elif i in range(10,14):
        CategoryStre.append('Minimal')
    elif i in range(14,21):
        CategoryStre.append('Moderate')
    elif i in range(21,28):
        CategoryStre.append('Severe')
    else:
        CategoryStre.append('Extremely Severe')
        
df['CategoryStre']= CategoryStre

In [ ]:
df.isnull().sum()
df.duplicated().sum()

In [ ]:
Y = df['CategoryStre']
X = df.drop(columns=['Q2A','Q4A','Q7A','Q9A','Q15A','Q19A','Q20A','Q23A','Q25A','Q28A','Q30A','Q36A','Q40A','Q41A','introelapse','testelapse', 'surveyelapse','engnat','CategoryStre','country', 'ScoresDepr' ,'ScoresStre','screensize','uniquenetworklocation','Q1A' ,'Q1I', 'Q1E', 'Q2I', 'Q2E', 'Q3A', 'Q3I', 'Q3E','Q4I','Q4E', 'Q5E', 'Q5A' ,'Q5I', 'Q6E', 'Q6A' ,'Q6I', 'Q7E' ,'Q7I', 'Q8E','Q8A' ,'Q8I', 'Q9E', 'Q9I', 'Q10E', 'Q10A', 'Q10I', 'Q11E', 'Q11A' ,'Q11I', 'Q12E', 'Q12A' ,'Q12I', 'Q13E', 'Q13A' ,'Q13I', 'Q14E', 'Q14A' ,'Q14I', 'Q15E', 'Q15I', 'Q16E', 'Q16A' ,'Q16I', 'Q17E', 'Q17A' ,'Q17I', 'Q18E', 'Q18A' ,'Q18I', 'Q19I', 'Q19E',  'Q20I', 'Q20E', 'Q21A', 'Q21I', 'Q21E', 'Q22A', 'Q22I','Q22E', 'Q23I','Q23E','Q24A','Q24I','Q24E', 'Q25I', 'Q25E', 'Q26A', 'Q26I', 'Q26E', 'Q27A', 'Q27I', 'Q27E', 'Q28I', 'Q28E', 'Q29A', 'Q29I', 'Q29E',  'Q30I', 'Q30E', 'Q31A', 'Q31I', 'Q31E', 'Q32A', 'Q32I', 'Q32E', 'Q33A', 'Q33I', 'Q33E', 'Q34A', 'Q34I', 'Q34E', 'Q35A', 'Q35I', 'Q35E', 'Q36I', 'Q36E', 'Q37A', 'Q37I', 'Q37E', 'Q38A', 'Q38I', 'Q38E', 'Q39A', 'Q39I', 'Q39E', 'Q40I', 'Q40E', 'Q41I', 'Q41E', 'Q42A', 'Q42I', 'Q42E'])
pd.set_option("display.max_columns", None)
X.head()

In [ ]:
#Pre-processing the data
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
print(X)

In [ ]:
# KNN Model for Stress Classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

# Create a KNN classifier with k (number of neighbors)
knn = KNeighborsClassifier(n_neighbors=95)

# Fit the model to the training data
knn.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = knn.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)

print('Accuracy of KNN Model - Stress: {:.5f}'.format(accuracy))
print('Confusion Matrix of KNN Model - Stress: \n', conf_matrix)
print('Confusion Matrix of KNN Model - Stress: \n', classif_report)

In [ ]:
# Naive Bayes Model for Stress Classification
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

clfNB = GaussianNB()

# Fit the model to the training data
clfNB.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = clfNB.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)

print('Accuracy of NB Model - Stress: {:.5f}'.format(accuracy))
print('Confusion Matrix of NB Model - Stress: \n', conf_matrix)
print('Classification Report of KNN Model - Stress: \n', classif_report)

In [ ]:
# Random Forest Classifier Model for Stress Classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=85)

clfRFC = RandomForestClassifier(n_estimators=95, random_state=85)

# Fit the model to the training data
clfRFC.fit(X_train, Y_train)

# Predict the classes of the testing set
Y_pred = clfRFC.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
classif_report = classification_report(Y_test, Y_pred)


print('Accuracy of RFC Model - Stress: {:.5f}'.format(accuracy))
print('Confusion Matrix of RFC Model - Stress: \n', conf_matrix)
print('Classification report of KNN Model - Stress: \n', classif_report)